In [15]:
import pandas as pd

In [16]:
'''
#1 – Acessar o site da Embrapa – Dados da Vitivinicultura, aba “produção” (http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_02) realizar a extração da base – arquivo salvo com o nome “Producao”

#2 – Acessar o site da Embrapa – Dados da Vitivinicultura, aba “exportação”, opção “vinhos de mesa” (http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_06) realizar a extração da base – arquivo salvo com o nome “ExpVinho”

#3 – Acessar o site da OIV (International Organisation of Vine and Wine), na opção “What we do” – “Statistics” e acessar a opção “DATABASE” https://www.oiv.int/what-we-do/data-discovery-report?oiv e filtrar:
1.1	Product = Wine
1.2	Variable = Consumption, Exports, Imports e Production
1.3	Year = 2008-2022
1.4	Continent = Africa, America, Asia, Europe, Oceania
1.5	Realizar a extração da base – arquivo salvo como “BD_OIV_Original”
'''

'\n#1 – Acessar o site da Embrapa – Dados da Vitivinicultura, aba “produção” (http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_02) realizar a extração da base – arquivo salvo com o nome “Producao”\n\n#2 – Acessar o site da Embrapa – Dados da Vitivinicultura, aba “exportação”, opção “vinhos de mesa” (http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_06) realizar a extração da base – arquivo salvo com o nome “ExpVinho”\n\n#3 – Acessar o site da OIV (International Organisation of Vine and Wine), na opção “What we do” – “Statistics” e acessar a opção “DATABASE” https://www.oiv.int/what-we-do/data-discovery-report?oiv e filtrar:\n1.1\tProduct = Wine\n1.2\tVariable = Consumption, Exports, Imports e Production\n1.3\tYear = 2008-2022\n1.4\tContinent = Africa, America, Asia, Europe, Oceania\n1.5\tRealizar a extração da base – arquivo salvo como “BD_OIV_Original”\n'

In [17]:
#1

df = pd.read_csv('/content/Producao.csv', sep=";")
# Excluir colunas 'id' e dos anos de 1970 a 2007 no DataFrames
df = df.drop(columns=['id'])
df = df.loc[:, ~df.columns.isin([str(year) for year in range(1970, 2008)])]

# Selecionar produtos a serem considerados

# Lista de produtos desejados
produtos_desejados = ['VINHO DE MESA', 'VINHO FINO DE MESA (VINÍFERA)']

# Filtra apenas as linhas com os produtos desejados
df = df[df['produto'].isin(produtos_desejados)]

# Aplicar melt no DF
df_melt = df.melt(id_vars=["produto"], var_name="Ano", value_name="Produção em Litros")


# Agrupa apenas por ano, somando a produção em litros
df_total_por_ano = df_melt.groupby('Ano')['Produção em Litros'].sum().reset_index()

# Exportar e baixar a base gerada em .csv para leitura no power bi
df_total_por_ano.to_csv('producao_total_ano.csv', index=False)


In [18]:
#2
# Abrir o arquivo .csv e salvá-lo em .xslx
# Importar os dados e gerar o DataFrame
df = pd.read_excel('/content/ExpVinho.xlsx')

# Excluir a coluna 'Id'
df = df.drop(columns=['Id'])

# Separar em duas bases
# Criar DataFrame exp_litros com as colunas de valores iniciais de cada ano
exp_litros = df.iloc[:, [0] + [i for i in range(1, len(df.columns), 2)]]
exp_litros.columns = ['País'] + [str(year) for year in range(1970, 2023)]

# Criar DataFrame exp_usd com as colunas da segunda coluna de valores de cada ano
exp_usd = df.iloc[:, [0] + [i for i in range(2, len(df.columns), 2)]]
exp_usd.columns = ['País'] + [str(year) for year in range(1970, 2023)]

# Excluir colunas dos anos de 1970 a 2007 nos DataFrames exp_litros e exp_usd
exp_litros = exp_litros.loc[:, ~exp_litros.columns.isin([str(year) for year in range(1970, 2008)])]
exp_usd = exp_usd.loc[:, ~exp_usd.columns.isin([str(year) for year in range(1970, 2008)])]

# Aplicar Melt nos DF
exp_litros_melt = exp_litros.melt(id_vars=["País"], var_name="Ano", value_name="Exportação em Litros")
exp_usd_melt = exp_usd.melt(id_vars=["País"], var_name="Ano", value_name="Exportação em Dólar")

# Importar e ler a base “De-Para Nomes países.xlsx”
base_paises = pd.read_excel('/content/De-Para Nomes países.xlsx')

#Ajustar o nome do país para a listagem padrão nas tabelas exp_usd_melt e exp_litros_melt

exp_usd_melt = pd.merge(exp_usd_melt, base_paises, left_on='País', right_on='PaÃ­s', how='inner')
exp_usd_melt = exp_usd_melt.drop(['Nome na base', 'Nome do País (Inglês)', 'País', 'PaÃ­s'], axis=1)
exp_usd_melt = exp_usd_melt[['Nome do País (Português)', 'Ano', 'Exportação em Dólar']]

exp_litros_melt = pd.merge(exp_litros_melt, base_paises, left_on='País', right_on='PaÃ­s', how='inner')
exp_litros_melt = exp_litros_melt.drop(['Nome na base', 'Nome do País (Inglês)', 'País', 'PaÃ­s'], axis=1)
exp_litros_melt = exp_litros_melt[['Nome do País (Português)', 'Ano', 'Exportação em Litros']]

# Exportar e baixar as bases geradas em .csv para leitura no power bi
exp_litros_melt.to_csv('exp_litros_melt.csv', index=False)
exp_usd_melt.to_csv('exp_usd_melt.csv', index=False)


In [19]:
#3
# Importar e ler o arquivo
bd_oiv = pd.read_excel('/content/BD_OIV_Original.xlsx')

#ajustar tipo da coluna “Year” para int
bd_oiv = bd_oiv.dropna(subset=['Year'])
bd_oiv['Year'] = bd_oiv['Year'].astype(int)

# Importar e ler a base “De-Para Nomes países.xlsx”
base_paises = pd.read_excel('/content/De-Para Nomes países.xlsx')

#Ajustar o nome do país para a listagem padrão nas tabelas exp_usd_melt e exp_litros_melt

bd_oiv = pd.merge(bd_oiv, base_paises, left_on='Region/Country', right_on='Nome do País (Inglês)', how='inner')

#Excluir colunas que não serão utilizadas
bd_oiv = bd_oiv.drop(['Region/Country', 'Nome do País (Inglês)', 'Nome na base', 'PaÃ­s', 'Unit'], axis=1)

# Renomeando as colunas
novos_nomes = {
    'Continent': 'Continente',
    'Product': 'Produto',
    'Variable': 'Variável',
    'Year': 'Ano',
    'Quantity': 'Quantidade em litros',
    'Nome do País (Português)': 'País'}

bd_oiv = bd_oiv.rename(columns=novos_nomes)


# Reorganizar a ordem das colunas

ordem_colunas = ['Continente', 'País', 'Variável', 'Ano', 'Quantidade em litros']
bd_oiv = bd_oiv[ordem_colunas]


# Ajustar quantidade de 1000 hl para litros

bd_oiv['Quantidade em litros'] = bd_oiv['Quantidade em litros'] * 100000

# Exportar e baixar a base gerada em .csv para leitura no power bi

bd_oiv.to_csv('bd_oiv.csv', index=False)


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
